In [187]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

import torch
import numpy as np

from method.learned_cn_method import CNModule, VecCNModule
from data import load_network
from util import Params

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Load Toy VecCNModule**

In [188]:
params = Params("../../experiments/disgenet/bio-pathways/dpp-10/learned_cn/cn_weighting/vec_lr_1e-0_ones/params.json")

In [189]:
params.d = 8

In [355]:
vec_cn_deep = VecCNModule(params, A)

**Manual Walk-Through**

In [268]:
A = torch.tensor([[0., 1., 0.],
                  [1., 0., 1.],
                  [0., 1., 0.]])

In [269]:
D = torch.sum(A, dim=1, keepdim=True)

In [270]:
D = torch.pow(D, -0.5)

In [271]:
A_left = D.view(1, -1) * A * D.view(-1, 1)

In [272]:
A_right = D.view(1, -1) * A

In [273]:
X = torch.tensor([[1., 0., 1.],
                  [1., 1., 0.]])

In [274]:
E = torch.tensor([1., 2., 3.]).view(1, 3)


In [279]:
AE = A_left*E
AE

tensor([[[0.5000, 0.0000, 1.5000],
         [0.0000, 2.0000, 0.0000],
         [0.5000, 1.4142, 0.0000]]])

In [276]:
AEA = torch.matmul(AE, A_right)

In [277]:
XAEA = torch.matmul(X, AEA)
XAEA

tensor([[2.8284, 0.0000, 2.8284],
        [1.4142, 2.0000, 1.4142]])

**Compute According to Exact Code from Class**

In [278]:
A = torch.tensor([[1., 0., 1.],
                  [0., 1., 0.],
                  [1., 1., 0.]]).numpy()
D = np.sum(A, axis=1, keepdims=True)

D = np.sum(A, axis=1, dtype=np.float)
D = np.power(D, -0.5)
D = torch.tensor(D, dtype=torch.float)

A = torch.tensor(A, dtype=torch.float)
A_left = torch.mul(torch.mul(D.view(1, -1), A), D.view(-1, 1))
A_right = torch.mul(D.view(1, -1), A)

E = torch.tensor([1., 2., 3.]).view(1, 1, 3)
X = torch.tensor([[1., 0., 1.],
                  [1., 1., 0.]])

X = torch.matmul(X, A_left)
X = torch.mul(X, E).squeeze()
X = torch.matmul(X, A_right)
X

tensor([[1.7678, 2.9142, 0.7071],
        [1.4142, 3.5000, 0.3536]])

**Full Tests**

In [186]:
params = Params("../../experiments/disgenet/bio-pathways/dpp-10/learned_cn/cn_weighting/lr_1e-0_ones/params.json")
_, adj, _ = load_network("../../" + params.ppi_network)